In [ ]:
import torch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from Util1 import DataManipulation

from joblib import dump, load

In [ ]:
# device selection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
if device == 'cuda':
    print(torch.cuda.get_device_name())

# 1. DATA PREPROCESSING

In [ ]:
Data = pd.read_excel(r'C:\Users\YUSIFOH\ERPE 394A\Huseyn_Yusifov_Homework_4\data\Geophysical_Logs_Well_1.xlsx', sheet_name='Well_1')
Data

## a) Handle Missing Values

In [ ]:
# missing data visualziation
m = DataManipulation(Data)
m.VisualizeMissingData()
m.MissingDataSummarizer()

In [ ]:
# removing missing data rows
Data_Manipulated = m.DropMissingData()
m.VisualizeMissingData()
m.MissingDataSummarizer()

## b) Remove Outliers

In [ ]:
# visualization of Cross and Box Plots to dedect outliers
m.CrossPlot()
m.BoxPlot()

In [ ]:
# Dedecting Outliers using Tukey's Method
for i in np.array(Data_Manipulated.columns):
    if not i in ['DEPTH','Medium Resistivity (RM)', 'Density (RHOB)']:
        data_tukey = Data_Manipulated[i]
        outliers = m.tukey_outliers(data_tukey, Data_Manipulated.index)[0]
        print(i, outliers)
        plt.hist(data_tukey, bins=50, color='gray', alpha=0.5)
        plt.xlabel(i)
        plt.ylabel('N')
        plt.title("Outlier Detection")
        plt.scatter(outliers, [0 for j in range(len(outliers))], color='red')
        plt.show()

Based on  the results from Tukey's outlier detection, we can be sure that the numbers greater than or equal to 999 are outliers

In [ ]:
for i in np.array(Data_Manipulated.columns):
    if not i in ['DEPTH','Medium Resistivity (RM)', 'Density (RHOB)']:
        Data_Manipulated = Data_Manipulated[Data_Manipulated[i]<999]

In [ ]:
# After removing outliers
m = DataManipulation(Data_Manipulated)
m.CrossPlot()
m.BoxPlot()

In [ ]:
m.MissingDataSummarizer()

## c) Scaling Dataset

In [ ]:
X = Data_Manipulated.drop(["Permeability (Perm)", "Water Saturation (SW)"], axis = 1)
y = Data_Manipulated[["Permeability (Perm)", "Water Saturation (SW)"]]

In [ ]:
# Data scaling
#X_scaler=StandardScaler()
#y_scaler=StandardScaler()
#
#X_scaled = X_scaler.fit(X)
#y_scaled = y_scaler.fit(y)
#
#dump(X_scaler, r'C:\Users\YUSIFOH\ERPE 394A\Huseyn_Yusifov_Homework_4\saved models\X_scaler.joblib')
#dump(y_scaler, r'C:\Users\YUSIFOH\ERPE 394A\Huseyn_Yusifov_Homework_4\saved models\y_scaler.joblib')

# load the saved model from a file
X_scaler = load(r'C:\Users\YUSIFOH\ERPE 394A\Huseyn_Yusifov_Homework_4\saved models\X_scaler.joblib')
y_scaler = load(r'C:\Users\YUSIFOH\ERPE 394A\Huseyn_Yusifov_Homework_4\saved models\y_scaler.joblib')

X_scaled = X_scaler.transform(X)
y_scaled = y_scaler.transform(y)

## d) Split the Dataset into training and validation sets

In [ ]:
X_scaled_train, X_scaled_test, y_scaled_train, y_scaled_test = train_test_split(X_scaled, y_scaled, test_size= 0.33, random_state=123)

In [ ]:
X_scaled_train = torch.tensor(X_scaled_train, dtype=torch.float).to(device = device)
X_scaled_test  = torch.tensor(X_scaled_test , dtype=torch.float).to(device = device)
y_scaled_train = torch.tensor(y_scaled_train, dtype=torch.float).to(device = device)
y_scaled_test  = torch.tensor(y_scaled_test , dtype=torch.float).to(device = device)

# 2. Model Architecture

In [ ]:
class NN_model(torch.nn.Module):

    def __init__(self,layers):
        super().__init__()

        self.layers = layers
        self.activation = torch.nn.Tanh()
        self.loss_function = torch.nn.MSELoss(reduction ='mean')
        self.linears = torch.nn.ModuleList([torch.nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])

        for i in range(len(layers)-1):
            torch.nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            torch.nn.init.zeros_(self.linears[i].bias.data)

    def forward(self,x):
        if torch.is_tensor(x) != True:
            x = torch.from_numpy(x)
        a = x.float()
        for i in range(len(self.layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)
        a = self.linears[-1](a)
        return a

    def loss(self, x, y_target ):
        L = self.loss_function(self.forward(x), y_target)
        return L

    def predict(self, x_test):
        y_pred = self.forward(x_test)
        y_pred = y_pred.cpu().detach().numpy()
        return y_pred

In [ ]:
input_dimension = X_scaled.shape[1]
output_dimension = y_scaled.shape[1]
layer_architectures = [[input_dimension, 10, 10, 10, output_dimension], [input_dimension, 15, 15, 15,15, output_dimension], [input_dimension, 20, 20, 20, output_dimension], [input_dimension, 25, 25, 25,25, output_dimension], [input_dimension, 30, 30, 30, output_dimension]]

In [ ]:
def R2(y_train_true, y_train_pred, y_test_true, y_test_pred, target):
    """R2 plot calculation"""
    # Get prediciton score in terms of R2
    r2_test = r2_score(y_test_true, y_test_pred)
    r2_train = r2_score(y_train_true, y_train_pred)

    # Plot parity plots
    fs = 9
    plt.figure(figsize=(5, 5))
    plt.suptitle('${}$ Parity Plot'.format(target), fontsize=fs)

    plt.subplot(2, 1, 1)
    plt.plot(y_test_true, y_test_pred, 'ro', label="Test: R2 = {}".format(round(r2_test, 3)))
    plt.plot([y_test_true.min(), y_test_true.max()], [y_test_true.min(), y_test_true.max()], 'k-.')
    plt.ylabel('Prediction, {}'.format(target), fontsize=fs)
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(y_train_true, y_train_pred, 'bo', label="Train: R2 = {}".format(round(r2_train, 3)))
    plt.plot([y_train_true.min(), y_train_true.max()], [y_train_true.min(), y_train_true.max()], 'k-.')
    plt.ylabel('Prediction, {}'.format(target), fontsize=fs)
    plt.legend()

    plt.xlabel('True, {}'.format(target), fontsize=fs)
    plt.tight_layout()
    plt.show()

# 3) Training

In [ ]:
# Adam Optimizer Parameters
learing_rate_Adam = [0.5e-3, 1e-3, 1.5e-3]
N_epochs_lst_Adam = [1000, 2000,  4000]
eps= 1e-08
weight_decay=0
amsgrad_lst = [False, True]

# LBFGS Oprimizer Parameters
learing_rate_LBFGS = [1.0, 1.5, 2.0]
N_epochs_lst_LBFGS = [1, 3, 5]
max_iter_lst=[100, 1000, 10000]
history_size_lst = [5, 10, 15]
line_search_fn = 'strong_wolfe'
tolerance_grad = 1e-07
tolerance_change = 1e-09

In [ ]:
def training_w_Adam(model, x_test, x_train, y_test_target, y_train_target, lr, N_epochs, eps, weight_decay, amsgrad, n, save_model):
    training_loss_list = []
    test_loss_list = []
    # Setting Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, eps=eps, weight_decay=weight_decay, amsgrad=amsgrad)
    # Training Loop
    for i in range(N_epochs):
        # initialize optimizer
        optimizer.zero_grad()
        # forward pass
        train_loss = model.loss(x_train, y_train_target)
        test_loss = model.loss(x_test, y_test_target)
         # collecting loss
        test_loss_list.append(test_loss.item())
        training_loss_list.append(train_loss.item())
        # print loss values
        print("# Epoch  = {} ".format(i), " | Train Loss = {}".format(train_loss.item()), " | Test Loss = {}".format(test_loss.item()))
        # backward pass
        train_loss.backward()
        optimizer.step()
    if save_model == True:
        torch.save(model.state_dict(), r"C:\Users\YUSIFOH\ERPE 394A\Huseyn_Yusifov_Homework_4\adam saved nn models\adam_model_{}.pth".format(n))
    return training_loss_list, test_loss_list

def training_w_LBFGS(model, x_test, x_train, y_test_target, y_train_target, lr, N_epochs, max_iter, history_size, tolerance_grad, tolerance_change, line_search_fn, n, save_model):
    training_loss_list = []
    test_loss_list = []
    # Setting Optimizer
    optimizer = torch.optim.LBFGS(model.parameters(),lr=lr, max_iter=max_iter, history_size = history_size, tolerance_grad=tolerance_grad, tolerance_change = tolerance_change, line_search_fn = line_search_fn)
    # Training Loop
    for i in range(N_epochs):
        def closure():
            # initialize optimizer
            optimizer.zero_grad()
            # forward pass
            train_loss = model.loss(x_train, y_train_target)
            test_loss = model.loss(x_test, y_test_target)
            # collecting loss
            training_loss_list.append(train_loss.item())
            test_loss_list.append(test_loss.item())
            print(" | Train Loss = {}".format(train_loss.item()), " | Test Loss = {}".format(test_loss.item()))
            # backward pass
            train_loss.backward()
            return train_loss
        optimizer.step(closure)
    if save_model == True:
        torch.save(model.state_dict(), r"C:\Users\YUSIFOH\ERPE 394A\Huseyn_Yusifov_Homework_4\lbfgs saved nn models\lbfgs_model_{}.pth".format(n))
    return training_loss_list, test_loss_list

## Model Using Adam:

In [ ]:
# Training and Testing By choosing random parameters: Adam
layers = [input_dimension, 15, 15, 15, 15, 15, output_dimension]
model = NN_model(layers).to(device=device)
lr = 1e-3
N_epochs = 10000
amsgrad = True
n = 0
train_loss, test_loss = training_w_Adam(model, X_scaled_test, X_scaled_train, y_scaled_test, y_scaled_train, lr, N_epochs, eps, weight_decay, amsgrad, n, save_model= False)

In [ ]:
y_scaled_test_pred = model.predict(X_scaled_test)
y_test_pred = y_scaler.inverse_transform(y_scaled_test_pred)
y_test_true = y_scaler.inverse_transform(y_scaled_test.cpu().numpy())

y_scaled_train_pred = model.predict(X_scaled_train)
y_train_pred = y_scaler.inverse_transform(y_scaled_train_pred)
y_train_true = y_scaler.inverse_transform(y_scaled_train.cpu().numpy())

R2(y_train_true[:, 0], y_train_pred[:, 0], y_test_true[:, 0], y_test_pred[:, 0], "Permeability")
R2(y_train_true[:, 1], y_train_pred[:, 1], y_test_true[:, 1], y_test_pred[:, 1], "Saturation")

## Model Using LBFGS:

In [ ]:
# Training and Testing By choosing random parameters: Adam
layers = [input_dimension, 15, 15, 15, 15, 15, output_dimension]
model1 = NN_model(layers).to(device=device)
lr = 1.0
N_epochs = 1
max_iter = 2000
history_size = 10
line_search_fn = 'strong_wolfe'
tolerance_grad = 1e-07
tolerance_change = 1e-09
train_loss, test_loss = training_w_LBFGS(model1, X_scaled_test, X_scaled_train, y_scaled_test, y_scaled_train, lr, N_epochs, max_iter, history_size, tolerance_grad, tolerance_change, line_search_fn, n, save_model = False)

In [ ]:
y_scaled_test_pred = model1.predict(X_scaled_test)
y_test_pred = y_scaler.inverse_transform(y_scaled_test_pred)
y_test_true = y_scaler.inverse_transform(y_scaled_test.cpu().numpy())

y_scaled_train_pred = model1.predict(X_scaled_train)
y_train_pred = y_scaler.inverse_transform(y_scaled_train_pred)
y_train_true = y_scaler.inverse_transform(y_scaled_train.cpu().numpy())

R2(y_train_true[:, 0], y_train_pred[:, 0], y_test_true[:, 0], y_test_pred[:, 0], "Permeability")
R2(y_train_true[:, 1], y_train_pred[:, 1], y_test_true[:, 1], y_test_pred[:, 1], "Saturation")

# FINE-TUNING

## Fine - Tuning : Adam

In [ ]:
n = 0
adam_model_parameters = {}
adam_train_test_losses = {}
for layers in  layer_architectures:
    for lr in learing_rate_Adam:
        for N_epochs in N_epochs_lst_Adam:
            for amsgrad in  amsgrad_lst:
                print("Model #{} is started".format(n))
                aux = []
                aux.append([layers, lr, N_epochs, amsgrad])
                adam_model_parameters[n] = aux
                model = NN_model(layers).to(device=device)
                adam_train_test_losses[n] = training_w_Adam(model, X_scaled_test, X_scaled_train, y_scaled_test, y_scaled_train, lr, N_epochs, eps, weight_decay, amsgrad, n, save_model= True)
                n+=1

## Evaluation of Model: Adam

In [ ]:
# Selecting Best Model and Plot Loss - Train
best_model = 0
best_model_loss = 1000
for i in adam_train_test_losses.keys():
    if adam_train_test_losses[i][1][-1] < best_model_loss:
        best_model = i
        best_model_loss = adam_train_test_losses[i][1][-1]

plt.title("Best Model: Adam")
plt.plot(adam_train_test_losses[best_model][0], color= 'r', label = "Train Loss")
plt.plot(adam_train_test_losses[best_model][1], color= 'g', label = "Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("MSE Loss")
plt.grid()
plt.show()

# Predict Using Best Model
layers = adam_model_parameters[best_model][0][0]
bst_model = NN_model(layers).to(device=device)
# Load The Best Model
bst_model.load_state_dict(torch.load(r"C:\Users\YUSIFOH\ERPE 394A\Huseyn_Yusifov_Homework_4\adam saved nn models\adam_model_{}.pth".format(best_model)))
y_scaled_test_pred = bst_model.predict(X_scaled_test)
y_test_pred = y_scaler.inverse_transform(y_scaled_test_pred)
y_test_true = y_scaler.inverse_transform(y_scaled_test.cpu().numpy())

y_scaled_train_pred = bst_model.predict(X_scaled_train)
y_train_pred = y_scaler.inverse_transform(y_scaled_train_pred)
y_train_true = y_scaler.inverse_transform(y_scaled_train.cpu().numpy())

R2(y_train_true[:, 0], y_train_pred[:, 0], y_test_true[:, 0], y_test_pred[:, 0], "Permeability")
R2(y_train_true[:, 1], y_train_pred[:, 1], y_test_true[:, 1], y_test_pred[:, 1], "Saturation")

print(" Adam Best Model Parameters: Model No = {}, Layers = {},  Learning Rate = {}, Number of Epochs = {}, AmsGrad = {}".format(best_model, adam_model_parameters[best_model][0][0], adam_model_parameters[best_model][0][1], adam_model_parameters[best_model][0][2], adam_model_parameters[best_model][0][3]))

## Fine - Tuning : LBFGS

In [ ]:
n = 0
lbfgs_model_parameters = {}
lbfgs_train_test_losses = {}
max_iter = 4000
N_epochs = 1
for layers in  layer_architectures:
    for lr in learing_rate_LBFGS:
        for history_size in  history_size_lst:
            print("Model #{} is started".format(n))
            aux = []
            aux.append([layers, lr, history_size])
            lbfgs_model_parameters[n] = aux
            model = NN_model(layers).to(device=device)
            lbfgs_train_test_losses[n] = training_w_LBFGS(model, X_scaled_test, X_scaled_train, y_scaled_test, y_scaled_train, lr, N_epochs, max_iter, history_size, tolerance_grad, tolerance_change, line_search_fn, n, save_model = True)
            n+=1

## Evaluation of Model: Adam

In [ ]:
# Selecting Best Model and Plot Loss - Train
best_model_ = 0
best_model_loss = 1000
for i in lbfgs_train_test_losses.keys():
    if np.mean(lbfgs_train_test_losses[i][1]) < best_model_loss:
        best_model = i
        best_model_loss = np.mean(lbfgs_train_test_losses[i][1])

plt.title("Best Model: LBFGS")
plt.plot(lbfgs_train_test_losses[best_model][0], color= 'r', label = "Train Loss")
plt.plot(lbfgs_train_test_losses[best_model][1], color= 'g', label = "Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("MSE Loss")
plt.grid()
plt.show()

# Predict Using Best Model
layers = lbfgs_model_parameters[best_model][0][0]
bst_model = NN_model(layers).to(device=device)
# Load The Best Model
bst_model.load_state_dict(torch.load(r"C:\Users\YUSIFOH\ERPE 394A\Huseyn_Yusifov_Homework_4\lbfgs saved nn models\lbfgs_model_{}.pth".format(best_model)))
y_scaled_test_pred = bst_model.predict(X_scaled_test)
y_test_pred = y_scaler.inverse_transform(y_scaled_test_pred)
y_test_true = y_scaler.inverse_transform(y_scaled_test.cpu().numpy())

y_scaled_train_pred = bst_model.predict(X_scaled_train)
y_train_pred = y_scaler.inverse_transform(y_scaled_train_pred)
y_train_true = y_scaler.inverse_transform(y_scaled_train.cpu().numpy())

R2(y_train_true[:, 0], y_train_pred[:, 0], y_test_true[:, 0], y_test_pred[:, 0], "Permeability")
R2(y_train_true[:, 1], y_train_pred[:, 1], y_test_true[:, 1], y_test_pred[:, 1], "Saturation")

print(" LBFGS Best Model Parameters: Model No = {}, Layers = {},  Learning Rate = {}, History Size = {}".format(best_model, lbfgs_model_parameters[best_model][0][0], lbfgs_model_parameters[best_model][0][1], lbfgs_model_parameters[best_model][0][2]))